In [51]:
import pandas as pd
import bson
import pymongo
from tqdm import tqdm
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags
import re

In [52]:
from collections import Counter
import pandas as pd
import numpy as np
from datetime import datetime

In [53]:
import pickle

In [54]:
df = pd.read_csv("twitter_accounts.csv", header=None)

In [55]:
df

,0
0,Junior_Stock
1,DogsButAlsoDogs
2,acheuse
3,catalinaphotog
4,InnoTechToday
...,...
112606,TigerMensSoccer
112607,BigRuckus7
112608,kaplanliaslan
112609,JPMorganAM


In [18]:
data = {}
keys_counter = []
db = db_client["tweets_pipeline_v2"].find({
                                           "username": {"$exists": True},
                                          }, no_cursor_timeout=True)
for i in tqdm(db, total = db.count()):
    account_name = i["username"]    
    if account_name not in data:
        data[account_name] = {}    
        data[account_name]["TOTAL"] = db_client["tweets_pipeline_v2"].find({"username": account_name}, no_cursor_timeout=True).count()

    if "last_date" not in data[account_name]:
        data[account_name]["last_date"] = i["tweet_timestamp"]
    else:
        if i["tweet_timestamp"] > data[account_name]["last_date"]:
            data[account_name]["last_date"] = i["tweet_timestamp"]
    
    status = None
    if 'status' in i:
        if not re.search('clone of ', i['status']) and not re.search('url-clone', i['status']):
            status = i['status']
        else:
            status = "clone"
#             _id = re.search('clone of (.*)$', i['status']).group(1)
#             graphic = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(_id)})
#             if graphic is not None and "status" in graphic: 
#                 status = graphic["status"]
        
    if status is not None and status not in data[account_name]:
        data[account_name][status] = 1
    else:
        if status is not None:
            data[account_name][status] += 1
            
            
    if "img_type" in i:
        image_type = i['img_type']
        if image_type not in data[account_name]:
            data[account_name][image_type] = 1
        else:
            data[account_name][image_type] += 1            
            
    if "graph_type" in i:
        graph_type = i['graph_type']            
        if graph_type not in data[account_name]:
            data[account_name][graph_type] = 1
        else:
            data[account_name][graph_type] += 1
                    

    if "technical" in i and i['technical'] == True:
        if "technical" not in data[account_name]:
            data[account_name]["technical"] = 1
        else:
            data[account_name]["technical"] += 1            
            
            
    keys_counter += list(i.keys())

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  
  0%|          | 0/3255004 [00:00<?, ?it/s]/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 3255004/3255004 [50:14<00:00, 1079.77it/s] 


In [4]:
pickle.dump( data, open( "data_new.p", "wb" ) )

NameError: name 'data' is not defined

In [5]:
data = pickle.load( open( "data_new.p", "rb" ) )

In [6]:
data

{'000kayakman': {'TOTAL': 29,
  'last_date': datetime.datetime(2020, 1, 26, 14, 37, 22),
  'picture': 29},
 '0010202': {'TOTAL': 84,
  'last_date': datetime.datetime(2018, 7, 3, 18, 58, 29),
  'picture': 41,
  'clone': 13,
  'graphic': 30,
  'line-area': 11,
  'column-vertical': 1,
  'deleted': 7,
  'technical': 3,
  'other-chart': 3,
  'scheme': 4,
  'infographic': 4,
  'tables': 1,
  'map': 3,
  'reserved': 5},
 '004nino': {'TOTAL': 8,
  'last_date': datetime.datetime(2020, 4, 3, 1, 41, 25),
  'picture': 7,
  'graphicone_search': 1,
  'graphic': 1},
 '007cigarjoe': {'TOTAL': 66,
  'last_date': datetime.datetime(2020, 1, 26, 21, 11, 42),
  'picture': 59,
  'scheme': 1,
  'infographic': 3,
  'map': 2,
  'tables': 1},
 '007thirty': {'TOTAL': 54,
  'last_date': datetime.datetime(2020, 1, 28, 18, 23, 16),
  'picture': 44,
  'tables': 1,
  'infographic': 4,
  'deleted': 3,
  'graphic': 3,
  'bubble-scatter': 1,
  'map': 2,
  'pie-donut': 1,
  'clone': 1},
 '00bb263079e14e7': {'TOTAL': 156,

In [68]:
accounts = pd.DataFrame.from_dict(data).T
accounts_full = pd.DataFrame.from_dict(data).T

In [69]:
# УДАЛЯЕМ ЛИШНИЕ КОЛОНКИ
excess_columns = ["infographic", "tables", "map", "scheme",
                  "reserved", "pipelined",
                    "SourceZero",                    
                    "oserrorreserved",
                    "screenreserved",
                    "screenpipelined",
                    "oserrorpipelined",
                    "techreserved",
                    "invalid_url",
                    "techpipelined",
                    "bloomreserved",
                    'line-area',
                    'column-vertical',
                    'other-chart',
                    'url-clone',
                    'bubble-scatter',
                    'pie-donut',
                    'bar-horizontal',
                    "non-chart",
                    "trasher",
                    None,
                    "invalid_img_url_graphicone_search"]

accounts = accounts[[col for col in accounts_for_remove.columns if col not in excess_columns]].copy()
print(accounts.shape)
accounts.head(5)

(100032, 11)


,TOTAL,last_date,picture,clone,graphic,deleted,technical,graphicone_search,graphicone_feed,deleted_from_analytics,validated
000kayakman,29,2020-01-26 14:37:22,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0010202,84,2018-07-03 18:58:29,41,13,30,7,3,NaN,NaN,NaN,NaN
004nino,8,2020-04-03 01:41:25,7,NaN,1,NaN,NaN,1,NaN,NaN,NaN
007cigarjoe,66,2020-01-26 21:11:42,59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
007thirty,54,2020-01-28 18:23:16,44,1,3,3,NaN,NaN,NaN,NaN,NaN


# REMOVE OLD ACCs OR WHICH NOT UPDATED SINCE 1, jan 2020

In [70]:
accounts["year"] = accounts["last_date"].apply(lambda x: x.year)
print(accounts.shape)
accounts = accounts[(accounts.year > 2019)]
print(accounts.shape)

(100032, 12)
(53510, 12)


# ACCOUNT HAS MINIMUM ONE GRAPHIC

In [71]:
accounts = accounts[accounts['graphic'] > 0]
print(accounts.shape)
accounts.head(10)

(23512, 12)


,TOTAL,last_date,picture,clone,graphic,deleted,technical,graphicone_search,graphicone_feed,deleted_from_analytics,validated,year
004nino,8,2020-04-03 01:41:25,7,NaN,1,NaN,NaN,1,NaN,NaN,NaN,2020
007thirty,54,2020-01-28 18:23:16,44,1,3,3,NaN,NaN,NaN,NaN,NaN,2020
00bb263079e14e7,156,2020-04-29 11:44:32,143,NaN,3,1,NaN,NaN,1,NaN,NaN,2020
020trader,29,2020-04-22 09:41:55,20,1,4,1,NaN,NaN,1,NaN,NaN,2020
0deflation,301,2020-05-28 01:13:23,27,76,260,108,NaN,36,3,6,3,2020
0l0ll00ll0l0,29,2020-01-02 08:08:27,13,6,14,5,NaN,NaN,NaN,NaN,NaN,2020
0risk0reward,217,2020-02-28 20:12:59,199,NaN,1,1,NaN,NaN,NaN,NaN,NaN,2020
0xbc90,6,2020-01-09 18:19:08,4,NaN,1,1,NaN,NaN,NaN,NaN,NaN,2020
0xdude,58,2020-04-17 19:13:12,47,2,2,NaN,NaN,NaN,NaN,NaN,NaN,2020
0xhiroki,25,2020-04-10 07:39:19,20,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,2020


In [72]:
def algo(x):
    result = (x.deleted + x.picture + x.technical) / (x.TOTAL)
    return result

In [73]:
def algo_main_2(x):
    if x.graphicone_search == 0 and x.graphicone_feed == 0 and x.technical != 0:
        result = (x.picture + x.technical) / (x.TOTAL)    
        return result
    else:
        return 0

In [74]:
def algoGraphicPicture(x):
    if x.picture + x.graphic != 0:
        result = (x.picture) / (x.picture + x.graphic)
        return result
    else:
        return 0

In [75]:
def algoGraphicTechnical(x):
    if x.graphic != 0:
        result = (x.technical) / (x.graphic)
        return result
    else:
        return 0

In [76]:
def algoGraphic(x):
    result = (x.graphic) / (x.TOTAL)
    return result

In [77]:
accounts = accounts.fillna(0)
accounts["ALGO_MAIN"] = accounts.apply(lambda x: algo(x), axis = 1)
accounts["ALGO_MAIN_2"] = accounts.apply(lambda x: algo_main_2(x), axis = 1)

accounts["ALGO_Picture"] = accounts.apply(lambda x: algoGraphicPicture(x), axis = 1)
accounts["ALGO_Tech"] = accounts.apply(lambda x: algoGraphicTechnical(x), axis = 1)
accounts["ALGO_Graphic"] = accounts.apply(lambda x: algoGraphic(x), axis = 1)

# ТЕХНИЧЕСКИХ ГРАФИКОВ МЕНЬШЕ 20% и вторым условием чистим мусор старых графиков

In [78]:
accounts = accounts[(accounts["ALGO_Tech"]<0.2) & (accounts["ALGO_MAIN_2"]<0.5)]
accounts.shape

(23027, 17)

In [79]:
accounts = accounts[accounts["ALGO_Picture"] <= 0.98]
accounts

,TOTAL,last_date,picture,clone,graphic,deleted,technical,graphicone_search,graphicone_feed,deleted_from_analytics,validated,year,ALGO_MAIN,ALGO_MAIN_2,ALGO_Picture,ALGO_Tech,ALGO_Graphic
004nino,8,2020-04-03 01:41:25,7,0,1,0,0,1,0,0,0,2020,0.875000,0.0,0.875000,0.0,0.125000
007thirty,54,2020-01-28 18:23:16,44,1,3,3,0,0,0,0,0,2020,0.870370,0.0,0.936170,0.0,0.055556
00bb263079e14e7,156,2020-04-29 11:44:32,143,0,3,1,0,0,1,0,0,2020,0.923077,0.0,0.979452,0.0,0.019231
020trader,29,2020-04-22 09:41:55,20,1,4,1,0,0,1,0,0,2020,0.724138,0.0,0.833333,0.0,0.137931
0deflation,301,2020-05-28 01:13:23,27,76,260,108,0,36,3,6,3,2020,0.448505,0.0,0.094077,0.0,0.863787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ztrk_muzaffer,31,2020-03-26 15:06:59,27,1,1,0,0,0,0,0,0,2020,0.870968,0.0,0.964286,0.0,0.032258
zuzuku,14,2020-04-30 06:37:22,13,1,1,0,0,0,0,0,0,2020,0.928571,0.0,0.928571,0.0,0.071429
zxc098xxx,30,2020-04-27 20:03:21,28,2,2,0,0,0,0,0,0,2020,0.933333,0.0,0.933333,0.0,0.066667
zz_zigurds,12,2020-01-13 18:46:02,10,0,1,1,0,0,0,0,0,2020,0.916667,0.0,0.909091,0.0,0.083333


In [80]:
search_feed_sum = accounts.graphicone_search.sum() + accounts.graphicone_feed.sum()
search_feed_sum

12566

In [81]:
# SO we loss less data

In [82]:
db = db_client["tweets_pipeline_v2"].find({"$or": [{'status': "graphicone_search"}, {'status': "graphicone_feed"}]
                                          }, no_cursor_timeout=True)

print(db.count())
print(search_feed_sum/db.count())

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """


14394
0.8730026399888843


In [83]:
pd.DataFrame(accounts.reset_index()["index"].values).to_csv("5_Jun_accounts.csv", header=None, index=None)

In [50]:
accounts.reset_index()[accounts.reset_index()["index"] == "_cosatu"]

,index,TOTAL,last_date,picture,clone,graphic,deleted,technical,graphicone_search,graphicone_feed,deleted_from_analytics,validated,year,ALGO_MAIN,ALGO_MAIN_2,ALGO_Picture,ALGO_Tech,ALGO_Graphic
175,_cosatu,37,2020-06-02 09:54:18,33,0,1,0,0,0,0,0,0,2020,0.891892,0.0,0.970588,0.0,0.027027


In [87]:
print(len(set(list(accounts_full.reset_index()["index"].values))-set(list(accounts.reset_index()["index"].values))))
print(len(set(list(accounts.reset_index()["index"].values))-set(list(accounts_full.reset_index()["index"].values))))

79028
0


In [91]:
pd.DataFrame(set(list(accounts_full.reset_index()["index"].values))-set(list(accounts.reset_index()["index"].values))).to_csv("removed_accs.csv", header=None, index=None)

In [66]:
len(df[0].values)

112611

In [84]:
.rese

,TOTAL,last_date,picture,clone,graphic,line-area,column-vertical,deleted,technical,other-chart,...,screenreserved,screenpipelined,oserrorpipelined,techreserved,non-chart,NaN,invalid_url,techpipelined,bloomreserved,invalid_img_url_graphicone_search
000kayakman,29,2020-01-26 14:37:22,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0010202,84,2018-07-03 18:58:29,41,13,30,11,1,7,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
004nino,8,2020-04-03 01:41:25,7,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
007cigarjoe,66,2020-01-26 21:11:42,59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
007thirty,54,2020-01-28 18:23:16,44,1,3,NaN,NaN,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zywavekoenig,17,2016-03-16 20:28:08,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zz_zigurds,12,2020-01-13 18:46:02,10,NaN,1,1,NaN,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzaman,58,2019-11-08 12:12:45,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzeerrooaa,11,2020-04-30 04:21:36,8,2,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
